In [110]:
import torch
import glob
import os
import librosa
import numpy as np


In [4]:
embedding = torch.randn(64, 10, 256)
embedding.shape[:2]

torch.Size([64, 10])

In [103]:
class DotDict(dict):
    """
    a dictionary that supports dot notation 
    as well as dictionary access notation 
    usage: d = DotDict() or d = DotDict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct=None):
        dct = dict() if not dct else dct
        for key, value in dct.items():
            if hasattr(value, 'keys'):
                value = Dotdict(value)
            self[key] = value

def load_params():
    with open('encoder/config/config.yaml', 'r') as f:
        params = yaml.load_all(f, Loader=yaml.FullLoader)
        dict_params = {}
        for doc in params:
            for k, v in doc.items():
                dict_params[k] = v
    return dict_params


hp = DotDict(load_params())


In [55]:
audio_path = glob.glob(os.path.dirname('/Users/bapt/Desktop/ENSAE/3ème Année/Advanced Machine Learning/LibriSpeech/train-clean-360/*/*'))                                      

In [67]:
speaker = audio_path[0]
chapters = os.listdir(speaker)
utterances = os.listdir(os.path.join(speaker, chapters[0]))
utterance = os.path.join(speaker, chapters[0], utterances[0])


In [84]:
utter, sr = librosa.core.load(utterance, sr=16000)

In [128]:
for j in range(10) if j == 5:
    print(j)

array([[  2560,  24576],
       [ 28160,  34304],
       [ 34816,  50688],
       [ 54272,  74752],
       [ 81408, 105472],
       [109568, 134144],
       [134656, 141824],
       [160768, 180736],
       [190464, 192512],
       [206336, 209408],
       [209920, 210432],
       [211968, 216064]])

In [133]:
utter_part = utter[interval[0]:interval[1]]
S = librosa.feature.melspectrogram(y=utter_part, sr=sr, n_fft=hp.data.nfft, hop_length=int(hp.data.hop*hp.data.sr), n_mels=hp.data.nmels)
S = librosa.power_to_db(S, ref=np.max)
S = S.astype(np.float32).T

In [136]:
S = librosa.core.stft(y=utter_part, n_fft=hp.data.nfft,
                                              win_length=int(hp.data.window * sr), hop_length=int(hp.data.hop * sr))
S = np.abs(S) ** 2
mel_basis = librosa.filters.mel(sr=hp.data.sr, n_fft=hp.data.nfft, n_mels=hp.data.nmels)
S = np.log10(np.dot(mel_basis, S) + 1e-6)

In [144]:
S[:, :hp.data.tisv_frame]

array([[-3.1404095 , -2.491884  , -1.9298481 , ..., -0.49076617,
        -1.3435485 , -2.6188498 ],
       [-2.325535  , -1.6984915 , -1.3745589 , ..., -0.5600696 ,
        -1.6265929 , -2.3679945 ],
       [-1.6866193 , -1.2038095 , -1.092519  , ..., -1.5648443 ,
        -2.034195  , -2.8860192 ],
       ...,
       [-5.7239413 , -5.6711307 , -5.5101867 , ..., -5.704185  ,
        -5.871908  , -5.818638  ],
       [-5.815968  , -5.665121  , -5.254036  , ..., -5.5803275 ,
        -5.8124013 , -5.8429875 ],
       [-5.827293  , -5.701777  , -5.3962607 , ..., -5.7845097 ,
        -5.9338746 , -5.895902  ]], dtype=float32)

In [147]:
utter_min_len = (hp.data.tisv_frame * hp.data.hop + hp.data.window) * hp.data.sr
utter_min_len

29200.0